In [105]:
#import docx2txt 
import spacy
import os
import re
import pandas as pd
# Using an english named entity recognition
# need to import the manual python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")
os.listdir("//ct-fs-02.childtrends.org/securedata/6669 WWK/Data exports/LIVE/04.27.2023/Targeted")
mcu = pd.read_csv("//ct-fs-02.childtrends.org/securedata/6669 WWK/Data exports/LIVE/04.27.2023/Targeted/MonthlyChildUpdate.csv")

In [115]:
def code_text(text): 
    # create funciton to id any named entities and collapse into TYPE: value 
  
    doc = nlp(str(text))
  
    doc_text = [el.text for el in doc.ents]
    doc_start = [el.start_char for el in doc.ents]
    doc_end = [el.end_char for el in doc.ents]
    doc_label = [el.label_ for el in doc.ents]
        
    if(len(doc_text)>0):
        df = pd.DataFrame(list(zip(doc_text, doc_start,doc_end,doc_label)))
        df.columns = ['text','start','stop','type']

        #remove Types taht we dont care about 
        remove_types = ['CARDINAL',
                        'DATE',
                        'QUANTITY','CARDINAL','ORDINAL',
                        'TIME',
                        'PERCENT','MONEY','WORK_OF_ART','GPE',
                        #"LOC",
                        "PRODUCT",
                        "NORP","ORG","LAW"]
        df_clean = df[~df['type'].isin(remove_types)]

        # remove specific words we know are not helpful 
        remove_words = ["wwk","recruiter"]
        df_clean = df_clean[~df_clean['text'].str.lower().isin(remove_words)]
        df_clean['res'] = df_clean['type'] + ": " + df_clean['text']
        return("; ".join(df_clean['res']))
    else: 
        return("")


# place and people names 

ot=list()
for col in mcu.columns: 
    if re.search("OT$",col):
        ot.append(col)
ot_coded = [el +"_coded" for el in ot]

mcu.head()

,IDCHILD,IDRECRUIT,SITE_ID,CODE,SITE_NAME,STATE,SUBMITTED,CHDATE,SUBMIT_DATE,LEGACY,...,CHSTMO,CHSTMOS,CHPERM_OTHER_coded,CHCFMTWHYOT_coded,MDOTHER_coded,CHRFOT_coded,CHSVOT_coded,FNUNTOT_coded,PPDOTHER_coded,OTHERPLACEMENT_coded
0,Child's WWK ID,Recruiter ID,Site ID,Textual abbreviation for site name,Site name,State,Record submitted,Month and year to which submission refers,Submission date (should be by 7th of subsequen...,Database generated - Record added before 2012,...,22a. Additional placement this month - Other -...,22a. Additional placement this month - Other (...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20507,3113,224,STAR3,OH - Stark 3 NCFR,Ohio,Yes,02/01/2023,03/01/2023,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20509,3113,224,STAR3,OH - Stark 3 NCFR,Ohio,Yes,02/01/2023,03/01/2023,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,22506,3113,224,STAR3,OH - Stark 3 NCFR,Ohio,Yes,02/01/2023,03/01/2023,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,26997,3113,224,STAR3,OH - Stark 3 NCFR,Ohio,Yes,02/01/2023,03/01/2023,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [116]:
ot

['CHCFMTWHYOT', 'CHRFOT', 'CHSVOT', 'FNUNTOT']

In [120]:
x = mcu[ot].apply(code_text,axis=1)
x.head()

0    
1    
2    
3    
4    
dtype: object

In [ ]:
text = 'Several attempts was made to see youth response pending from foster dad '
doc = nlp(text)
doc_text = [el.text for el in doc.ents]
doc_start = [el.start_char for el in doc.ents]
doc_end = [el.end_char for el in doc.ents]
doc_label = [el.label_ for el in doc.ents]
df = pd.DataFrame(list(zip(doc_text, doc_start,doc_end,doc_label)))
df

In [83]:
doc = nlp(text)
doc_text = [el.text for el in doc.ents]
doc_text

[]

In [68]:
mcu[var + "_coded"].unique()

array(['', 'DATE: the entire month',
       'Several attempts was made to see youth; response pending from foster dad. ',
       'Child "no showed" due to sickness', 'PERSON: Recruiter',
       'Child is placed out of state. ', 'Youth AWOL', 'Child is awol',
       'PERSON: Child Refused', 'PERSON: Shannan',
       'PERSON: Recruiter; DATE: annual', 'Child is AWOL. ',
       'DATE: last week of the month',
       'PERSON: Belmont Pines; PERSON: Belmont Pines 2/21/23; PERSON: Greg',
       'PERSON: Jonyce',
       'Because I am working close with my supervisor and slowly growing my caseload. ',
       'DATE: the middle of the month; DATE: the first full week of March',
       'Child is AWOL ',
       'WWKR continues to make attempts to meet with youth without success. ',
       "Recruiter scheduled visit with foster dad to do a zoom visit with the child's sister. Foster dad did not follow through with the zoom visit and stopped communicating.",
       'DATE: February', 'safety concerns 

In [58]:
mcu['CHCFMTWHYOT_coded'].iloc[2].text=='nan'

True

In [62]:
test = mcu.head()
test2= test[test[var]!=""]
test2
test[var].astype('str').apply(code_text)





0        
1    None
2    None
3    None
4    None
Name: CHCFMTWHYOT, dtype: object

In [ ]:
# Append all to list 
check = []
names = []
for i in range(0,len(clean_files)-1):
    print(clean_files[i])
    doc = nlp(text[i])
    doc_text = [el.text for el in doc.ents]
    doc_start = [el.start_char for el in doc.ents]
    doc_end = [el.end_char for el in doc.ents]
    doc_label = [el.label_ for el in doc.ents]

    df = pd.DataFrame(list(zip(doc_text, doc_start,doc_end,doc_label)))
    df.columns = ['text','start','stop','type']

    #remove Types taht we dont care about 
    remove_types = ['CARDINAL','DATE','QUANTITY','CARDINAL','ORDINAL','TIME','PERCENT','MONEY','WORK_OF_ART','GPE',"LOC","NORP","ORG","LAW"]
    df_clean = df[~df['type'].isin(remove_types)]

    # remove specific words we know are not helpful 
    remove_words = ["moa","mou",'name child','school','max','isc-dc','ssc','isc dc','nij','ta','state superintendent','the advisor board','safe','metro','webinar','student council','mou','child trends', 'ice','ohr','cap','highschool','covid','npss','kinda','hacking school discipline','mm-hmm','ms.','powerpoint','mrs.','pd',"gotcha",'merry christmas',"ap",'dcps','dc','she\'s--','Where\'d the day go','yada','safe school certification','a google doc','bye-bye','girl circle']
    df_clean = df_clean[~df_clean['text'].str.lower().isin(remove_words)]

    # remove deideintified names (School A etc)
    df_clean = df_clean[pd.isnull(df_clean['text'].apply(lambda x: re.match("School [A-Z]\]|School [A-Z]$|\[TA.*",x)))]

    # remove checkpoint references and improving school climate
    df_clean = df_clean[pd.isnull(df_clean['text'].apply(lambda x: re.match("Checkpoint|.*Improving School Climate|.*Safe School Certification.*|.*CAB.*|.*Certification Advisory Board.*|.*Safe School.*|.* community advisory board",x)))]

    # dashes arent handled well 
    df_clean = df_clean[pd.isnull(df_clean['text'].apply(lambda x: re.match(".*--",x)))]
    print(df_clean)
    
    output = df_clean[['text']].drop_duplicates()
    print("++++++++++++++++++++++++++++++++++")
    # only output results if named entities are detected
    if output.shape[0]>0:
        check.append(output)
        names.append(clean_files[i])


In [16]:
mcu['CHCFMTWHYOT'].unique()

array(['2a. Reason unable to see child - Other (specify) - Added 9/2007',
       nan, "Child's recruiter is on medical leave.",
       "Child's recruiter is on medical leave",
       'Child has been AWOL for the entire month',
       'Several attempts was made to see youth; response pending from foster dad. ',
       'Child "no showed" due to sickness',
       'Recruiter attempted to contact pre-adoptive parent three times. Recruiter left voicemails on 2/1 and 2/14. Recruiter sent a text on 2/7. Recruiter informed caseworker that she was unable to get into contact with pre-adoptive parent on 2/16. No response.',
       'Child is placed out of state. ', 'Youth AWOL', 'Child is awol',
       'Cw forgot to let recruiter know when she was scheduled to meet with the children.  Foster mother is reportedly "picky" about visits.  Recruiter reached out to foster mother to schedule a visit, but she never got back to recruiter.',
       'Child Refused visit',
       'Scheduled two visits but Shan